#### Base Backtesting

In [1]:
# Backtesting Python Script
# (c) MOSTORO
# QSOCIETY


import pandas as pd
from arctic import Arctic
from arctic.date import DateRange
import matplotlib.pyplot as plt
import numpy as np

'''Base class for event-based backtesting of trading strategies'''


def __init__():
start = '2020-11-17 06:00:00'  #start date for data selection
end = '2020-11-20 18:00:00' #end date for data selection
symbol = 'MES' #Financial instrument to be used
amount = 3000 #amount for invest
position = 0 #0: out of market, 1: long, -1: short
margin = True #Validation to trade according to the margin
instruments = pd.read_csv(r'G:\My Drive\11 SHARING\Carpe Diem Society\Trading\Python\MOSTORO\Code\Data\Instruments Specifications\instruments.csv').set_index('symbol')  # windows
instrument = instruments.loc[symbol]
init_margin = float(instrument.init_margin)
maint_margin = float(instrument.maint_margin)
tick_size = float(instrument.tick_size)
comm_value = float(instrument.comm_value)
slippage = float(instrument.tick_size * 2) #The idea is to simulate a mistake acording to the entry price
point_value = float(instrument.point_value)
margin_call_signal = 0
profit = pd.DataFrame({'Instrument': [0], 'Direction': [0], 'Contracts': [0], 'Commision':
                            [0], 'Price': [0], 'Stop': [0], 'Target': [0], 'Time': [0],
                            'Position': [0], 'Profit': [0]})
stop_loss = 0
target = 0

# Getting data:
'retrieves and prepares the base data set'
print('Getting the data...')
store = Arctic('161.35.0.227')
library = store['Futures_Historical_Ticks']
item = library.read(symbol, date_range=DateRange(start, end))
data = item.data
data = data['Last'].resample('1Min').ohlc()
print('Loaded data')

def get_date(bar): #returns the date and price for the given bar
    'Return date for bar'
    global date 
    date = data.index[bar].date()
    global weekday 
    weekday = date.weekday()
    global hour 
    hour = data.index[bar].time()

def calculate_margins(price=0, contracts=0): #Calculate the respectives margins
    'Calcultes the margins, according with each market'
    global init_margin
    global maint_margin
    init_margin = init_margin *  contracts
    maint_margin = maint_margin * contracts

def calculate_commision(contracts): #calculate commision per trade
    'generates the commission'
    commision = comm_value * contracts
    return commision

def place_order(direction, price, contracts, bar): #places a buy or sell order
    'Place a buy or sell order'
    global position
    global init_margin
    global profit
    global amount
    global commision
    if position == 0:
        global init_margin
        global profit
        calculate_margins(contracts=contracts)
        if amount >= init_margin:
            if direction == 'Long':
                price = price
                position += contracts  # contracts
                commision = calculate_commision(contracts=contracts)
                trade = {
                    'Instrument': symbol, 'Direction': direction,  'Contracts': contracts,
                    'Commision': commision, 'Price': price, 'Stop': price - stop_loss,
                    'Target': price + target, 'Time': data.index[bar], 'Position': position}
            elif direction == 'Short':
                price = price
                position -= contracts
                commision = calculate_commision(contracts=contracts)
                trade = {
                    'Instrument': symbol, 'Direction': direction,  'Contracts': contracts,
                    'Commision': commision, 'Price': price, 'Stop': price + stop_loss,
                    'Target': price - target, 'Time': data.index[bar], 'Position': position}
            profit = profit.append(trade, ignore_index=True)
            trades()
        else:
            print('Amount is not enough')
            metrics()

    else:
        if direction == 'Short':
            price = price
            position -= contracts
        elif direction == 'Long':
            price = price
            position += contracts
        trade = {
            'Instrument': symbol, 'Direction': direction,  'Contracts': contracts, 'Commision': commision,
            'Price': price, 'Stop': 0, 'Target': 0, 'Time': data.index[bar], 'Position': position}
        profit = profit.append(trade, ignore_index=True)
        trades()


def trades(): #Calculate the las profit of the last trade
    
    global amount
    if profit['Position'].iloc[-1] == 0:
        if profit['Position'].iloc[-1] - profit['Position'].iloc[-2] == 1:
            profit.loc[profit.index[-1], 'Profit'] = ((profit['Price'].iloc[-2] - profit['Price'].iloc[-1]) * point_value) - profit['Commision'].iloc[-2] -  profit['Commision'].iloc[-1]
            
        elif profit['Position'].iloc[-1] - profit['Position'].iloc[-2] == -1:
            profit.loc[profit.index[-1], 'Profit'] = ((profit['Price'].iloc[-1] - profit['Price'].iloc[-2]) * point_value) - profit['Commision'].iloc[-1] -  profit['Commision'].iloc[-2]
            
    else:
        profit.loc[profit.index[-1], 'Profit'] = 0
    amount +=  profit.loc[profit.index[-1], 'Profit']
    

def margin_call(i): #Margin call if actual profit is upper than maintaince margin
    'Current profit according with bar'
    global amount
    current_profit = 0
    if profit['Position'].iloc[i] == 1:
        current_profit = (data.close.iloc[bar] - profit.Price.iloc[i]) * point_value
        if amount + current_profit <= maint_margin:
            margin_call_signal += 1
            print('Margin Call!')
    elif profit['Position'].iloc[i] == -1:
        current_profit = (profit.Price.iloc[i] - data.close.iloc[bar]) * point_value
        if amount + current_profit <= maint_margin:
            margin_call_signal += 1
            print('Margin call!')
            
def metrics():#Items for evaluate the performance
    
    print('-' * 50)
    #Net Profit
    profit_copy = profit.copy()
    profit_copy['Net_Profit'] = profit_copy['Profit'].cumsum()
    print('Net Profit: {}'.format(profit_copy['Net_Profit'].iloc[-1].round(2)))

    #profit factor
    profit_factor = profit_copy['Profit'][profit_copy['Profit'] > 0].sum().round(2) / abs(profit_copy['Profit'][profit_copy['Profit'] < 0]).sum().round(2)
    print('profit_factor: {:.2f}'.format(profit_factor))

    #Max Drawdown
    max_drawdown = round(((profit_copy['Net_Profit'] / profit_copy['Net_Profit'].cummax() - 1).fillna(0).replace([-np.inf]).dropna()).min(), 2)
    print('max_drawdown: {}'.format(max_drawdown))

    #Sharpe Ratio
    sharpe_ratio = profit_copy['Profit'].mean() / profit_copy['Profit'].std()
    annualize_sharpe_ratio = (252 ** 0.5) * sharpe_ratio #252 trading days in the year

    print('sharpe_ratio: {:.2f}'.format(sharpe_ratio))
    print('annualize_sharpe_ratio: {:.2f}'.format(annualize_sharpe_ratio))

    #Sortino Ratio
    sortino_ratio = round(profit_copy['Profit'].mean() * 100 / profit_copy['Profit'][profit_copy['Profit'] < 0].std() * 100, 2)
    annualize_sortino_ratio = (252 ** 0.5) * sortino_ratio #252 trading days in the year

    print('sortino_ratio: {:.2f}'.format(sortino_ratio))
    print('annualize_sortino_ratio: {:.2f}'.format(annualize_sortino_ratio))


    #Ulcer index
    ulcer_index = ((((profit_copy['Net_Profit'] / profit_copy['Net_Profit'].cummax() - 1).fillna(0).replace([-np.inf]).dropna()) ** 2).sum() / profit_copy['Profit'].count()) ** 0.5
    annualized_ulcer_index = (252 ** 0.5) * ulcer_index

    print('ulcer_index: {:.2f}'.format(ulcer_index))
    print('annualized_ulcer_index: {:.2f}'.format(annualized_ulcer_index))

    #Percentaje profitable
    percentaje_profitable = profit_copy['Profit'][profit_copy['Profit'] > 0].count() / profit_copy['Position'][profit_copy['Position'] != 0].count() * 100

    print('percentaje_profitable: {:.2f}'.format(percentaje_profitable))
    print('-' * 50)

    #Equity Curve
    equity_curve = profit_copy[profit_copy['Position'] == 0]['Net_Profit'].plot(figsize=(12,8), title='Equity Curve')
    plt.show()

C:\Users\toro9\Anaconda3\lib\site-packages\arctic\store\_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


Getting the data...


LibraryNotFoundException: Library Futures_Historical_Ticks was not correctly initialized in <Arctic at 0x2abf195a348, connected to MongoClient(host=['161.35.0.227:27017'], document_class=dict, tz_aware=False, connect=True, maxpoolsize=4, sockettimeoutms=600000, connecttimeoutms=2000, serverselectiontimeoutms=30000)>.
Reason: ServerSelectionTimeoutError("161.35.0.227:27017: timed out, Timeout: 30.0s, Topology Description: <TopologyDescription id: 603eda9fa077224b268fc6e6, topology_type: Single, servers: [<ServerDescription ('161.35.0.227', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('161.35.0.227:27017: timed out')>]>"))

##### ------------------------------------